In [3]:
import pandas as pd

df = pd.read_csv('../data/df_final.csv')
df


,Patient_ID,Age,Gender,Sleep_Disorder_Type,AHI_Score,SaO2_Level,OCR_Extracted_Text,Diagnosis_Confirmed,OCR_Text_Limpio,OCR_Text_Tokens,...,disorder,insomnia,leg,narcolepsy,obstructive,patient,restless,sleep,syndrome,with
0,P0001,54,Female,Restless Leg Syndrome,44.11,77.59,Patient P0001 diagnosed with Restless Leg Synd...,1,patient p diagnosed with restless leg syndrome,"['patient', 'p', 'diagnosed', 'with', 'restles...",...,0.000000,0.000000,0.538616,0.0,0.000000,0.207909,0.538616,0.000000,0.538616,0.207909
1,P0002,17,Female,Restless Leg Syndrome,33.55,76.22,Patient P0002 diagnosed with Restless Leg Synd...,1,patient p diagnosed with restless leg syndrome,"['patient', 'p', 'diagnosed', 'with', 'restles...",...,0.000000,0.000000,0.538616,0.0,0.000000,0.207909,0.538616,0.000000,0.538616,0.207909
2,P0003,74,Male,Insomnia,12.91,89.53,Patient P0003 diagnosed with Insomnia.,1,patient p diagnosed with insomnia,"['patient', 'p', 'diagnosed', 'with', 'insomnia']",...,0.000000,0.806607,0.000000,0.0,0.000000,0.341265,0.000000,0.000000,0.000000,0.341265
3,P0004,63,Male,Restless Leg Syndrome,33.73,85.65,Patient P0004 diagnosed with Restless Leg Synd...,1,patient p diagnosed with restless leg syndrome,"['patient', 'p', 'diagnosed', 'with', 'restles...",...,0.000000,0.000000,0.538616,0.0,0.000000,0.207909,0.538616,0.000000,0.538616,0.207909
4,P0005,23,Female,Insomnia,14.84,88.70,Patient P0005 diagnosed with Insomnia.,1,patient p diagnosed with insomnia,"['patient', 'p', 'diagnosed', 'with', 'insomnia']",...,0.000000,0.806607,0.000000,0.0,0.000000,0.341265,0.000000,0.000000,0.000000,0.341265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,P0996,23,Female,Insomnia,41.57,83.14,Patient P0996 diagnosed with Insomnia.,1,patient p diagnosed with insomnia,"['patient', 'p', 'diagnosed', 'with', 'insomnia']",...,0.000000,0.806607,0.000000,0.0,0.000000,0.341265,0.000000,0.000000,0.000000,0.341265
996,P0997,49,Female,No Disorder,42.12,84.85,Patient P0997 diagnosed with No Disorder.,1,patient p diagnosed with disorder,"['patient', 'p', 'diagnosed', 'with', 'disorder']",...,0.895138,0.000000,0.000000,0.0,0.000000,0.257376,0.000000,0.000000,0.000000,0.257376
997,P0998,75,Male,Obstructive Sleep Apnea,18.66,90.08,Patient P0998 diagnosed with Obstructive Sleep...,1,patient p diagnosed with obstructive sleep apnea,"['patient', 'p', 'diagnosed', 'with', 'obstruc...",...,0.000000,0.000000,0.000000,0.0,0.524444,0.241437,0.000000,0.524444,0.000000,0.241437
998,P0999,55,Female,Insomnia,31.60,95.04,Patient P0999 diagnosed with Insomnia.,1,patient p diagnosed with insomnia,"['patient', 'p', 'diagnosed', 'with', 'insomnia']",...,0.000000,0.806607,0.000000,0.0,0.000000,0.341265,0.000000,0.000000,0.000000,0.341265


### Prueba Modelo de predicción

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Supongamos que la columna objetivo es 'Diagnosis_Confirmed'
X = tfidf_df
y = df['Diagnosis_Confirmed']  # Usamos la columna correcta

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Predecir
y_pred = clf.predict(X_test)

# Reporte
print(classification_report(y_test, y_pred))


NameError: name 'tfidf_df' is not defined